<a href="https://colab.research.google.com/github/dnguyend/MiscCollection/blob/main/colab/EinsteinRicciStiefel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ricci Tensor and Einstein metrics on Stiefel manifold $St_{p,n}$ with left-invariant diagonal metrics
We show a numerical example of a simple formula for the Ricci tensor, when the metric on $St{p, n}$ is given by $\langle (A, B), (A, B)\rangle_P = \sum_{i<j}a_{ij}^2t_{ij} +\sum_{i=1}^{n-p}\sum_{j=1}^pb_{ij}^2s_j$, where a tangent vector to the Stiefel manifold is represented by two matrices $A$ adn $B$, with $A$ anti-symmetric in $R^{p\times p}$ and $B\in R^{(n-p)\times p}$.

The Ricci tensor is given by $Ric(A, A) = \sum_{i<j}a_{ij}^2c_{ij} + \sum b_{ij}^2f_j$ with
$$c_{ij} = n-2+ \frac{n-p}{2}\frac{t_{ij}^2 -s_i^2-s_j^2}{s_is_j} + \frac{1}{2}\sum_{l\neq i, l\neq j}\frac{t_{ij}^2-t_{il}^2-t_{jl}^2}{t_{il}t_{jl}}
$$
$$f_j = n-2 + \frac{1}{2}\sum\frac{s_j^2 - t_{jl}^2 - s_l^2}{s_l t_{jl}}$$
to solve for Einstein metric on $St_{p, n}$

We can run the codes cell by cell but we can just follow the code without the run. Some cell does not return values as it may contain definition only.

We keep the metric as either a vector $TS$ of size $p(p-1)/2+p$, or a pair of $p\times p$ symmetric matrix with diagonal zero and a diagonal metric size $p$.

The metric is the Einstein metric in the paper $St_{5,7}$ 

In [4]:
import numpy as np
import contextlib
from scipy.optimize import minimize


def makeTmat(Tdiag):
    p = .5 * (1 + np.sqrt(1 + 8 * len(Tdiag)))
    p = int(np.round(p))

    Tmat = np.zeros((p, p))
    tx = np.triu_indices(p, 1)
    Tmat[tx] = Tdiag
    Tmat[tx[1], tx[0]] = Tdiag
    return Tmat


def makeTSmat(v, p):
    nupa = p*(p-1)//2    
    
    uu = np.zeros((p+1, p+1))
    uu[:p, :p] = makeTmat(v[:nupa])
    uu[:-1, -1] = v[nupa:]
    uu[-1, :-1] = v[nupa:]
    return uu

def RicciCoeff(TS, n, p):
    nupa = p*(p-1)//2
    Tdiag = TS[:nupa]
    uu = makeTmat(Tdiag)
    uui = makeTmat(1/Tdiag)
    Sdiag = TS[nupa:]

    tx = np.triu_indices(p , 1)

    kfA = np.zeros(nupa)
    for ss in range(nupa):
        i, j = tx[0][ss], tx[1][ss]
        kfA[ss] += n - 2
        kfA[ss] += -0.5*(n-p)*(Sdiag[i]/Sdiag[j] + Sdiag[j]/Sdiag[i])
        kfA[ss] += 0.5*(n-p)*uu[i, j]*uu[i, j]/Sdiag[i]/Sdiag[j]

        for aa in range(p):
            if aa != j and aa != i:
                kfA[ss] += -0.5*(uu[i, aa]/uu[j, aa] + uu[j, aa]/uu[i, aa])
                kfA[ss] += +0.5*uu[i, j]*uu[i, j]/uu[j, aa]/uu[i, aa]

    kfB = np.zeros((n-p, p))
    for i in range(n-p):
        for j in range(p):
            kfB[i, j] += n - 2                    
            kfB[i, j] += -0.5*((1/Sdiag)*uu)[j, :].sum()
            kfB[i, j] += 0.5*Sdiag[j]*Sdiag[j]*(1/Sdiag*uui)[j, :].sum()
            kfB[i, j] += -0.5*(Sdiag*uui)[j, :].sum()                

    return np.concatenate([kfA, kfB.reshape(-1)])


@contextlib.contextmanager
def printoptions(*args, **kwargs):
    original = np.get_printoptions()
    np.set_printoptions(*args, **kwargs)
    try:
        yield
    finally:
        np.set_printoptions(**original)


def pprint(mat, precision=4):
    """pretty print numpy matrix
    """
    with printoptions(precision=precision, suppress=True):
        print(mat)


There are 3 metrics found - some matches with Jensen and Arvanitoyeorgos et. al. and there is a new metric, corresponding to a partition $(1, 1, 3)$ of $5$. Note in the output, the right-most column (and bottom row) corresponds to $s$, the $p\times p$ top-left block correspond to $t$.

In [10]:
def verify_stf_75_113_2():    
    TS = np.array(
        [4.15568173, 3.86531131, 3.8653113 , 3.8653113,
          1.78072524, 1.78072524, 1.78072524,     
          0.4995625 , 0.49956253,  0.4995625,
        1.        , 4.09222011, 3.79800143, 3.79800143, 3.79800141])
    
    n = 7
    p = 5
    print(TS.shape, n, p)
    Tmat = makeTSmat(TS, p)
    rcf = RicciCoeff(TS, n, p)[:p*(p+1)//2]
    print("metric coefficients")
    pprint(Tmat)

    print("ricci tensor coefficients")
    pprint(makeTSmat(rcf, p))

    lbd = rcf/TS
    print("Ratios")
    pprint(makeTSmat(lbd, p))
    
    print('Ratio =%.5f standard deviation = %f' % (lbd.mean(), np.std(lbd)))
verify_stf_75_113_2()

(15,) 7 5
metric coefficients
[[0.     4.1557 3.8653 3.8653 3.8653 1.    ]
 [4.1557 0.     1.7807 1.7807 1.7807 4.0922]
 [3.8653 1.7807 0.     0.4996 0.4996 3.798 ]
 [3.8653 1.7807 0.4996 0.     0.4996 3.798 ]
 [3.8653 1.7807 0.4996 0.4996 0.     3.798 ]
 [1.     4.0922 3.798  3.798  3.798  0.    ]]
ricci tensor coefficients
[[0.     4.7001 4.3717 4.3717 4.3717 1.131 ]
 [4.7001 0.     2.014  2.014  2.014  4.6283]
 [4.3717 2.014  0.     0.565  0.565  4.2955]
 [4.3717 2.014  0.565  0.     0.565  4.2955]
 [4.3717 2.014  0.565  0.565  0.     4.2955]
 [1.131  4.6283 4.2955 4.2955 4.2955 0.    ]]
Ratios
[[0.    1.131 1.131 1.131 1.131 1.131]
 [1.131 0.    1.131 1.131 1.131 1.131]
 [1.131 1.131 0.    1.131 1.131 1.131]
 [1.131 1.131 1.131 0.    1.131 1.131]
 [1.131 1.131 1.131 1.131 0.    1.131]
 [1.131 1.131 1.131 1.131 1.131 0.   ]]
Ratio =1.13100 standard deviation = 0.000000
